# Simulated Annealing

Simulated annealing uses an analogy to physical annealing. The higher the "heat" the more "flexible" the algorithm: we look at our current position and consider a nearby position: if the nearby position is better, always go to it. If it's worse, we have a probability of going to it anyway, which is higher the higher the "heat". As the algorithm progresses, it "cools" and so becomes more and more rigid in terms of only going to strictly better solutions. This allows it to eventually converge on the global optimum (cool behaviour) while still escaping local optima (hot behaviour).

In [1]:
import random as r
import numpy as np

## Objective Functions

In this notebook we'll test the semi-empirical mass formula. It's a formula from physics which gives us the binding energy of a nucleus given its proton number, Z, and neutron number, N. Since physics is effectively trying to maximise the binding energy per nucleon, we have a fun optimization problem if we divide the energy predicted by the SEMF by Z + N. You can play around with physics and see which isotopes would be stable if you changed, say, the Coulomb term! 

SEMF is also useful since it has a known best result of Nickel-62: Z = 28, N = 34. The closer the output is to this, the better our algorithm is doing. Another very stable isotope is Iron-56: Z = 26, N = 30, so if we get a result close to this the algorithm has done well, too.

More about the SEMF here: https://en.wikipedia.org/wiki/Semi-empirical_mass_formula

If you want a different objective function, check out Objectives.ipynb.

In [2]:
def SEMF(
    hypothesis, 
    volumeConstant = 15.8,
    surfaceConstant = 17.8,
    coulombConstant = 0.711,
    asymmetryConstant = 23.7,
    pairingConstant = 11.18
):
    
    #Physical info from hypothesis
    Z = int(hypothesis[0]) #Proton number
    N = int(hypothesis[1]) #Neutron number
    A = Z + N              #Nucleon number
    
    #Calculate each term
    volumeTerm = volumeConstant * A
    surfaceTerm = -surfaceConstant * A ** (2/3)
    coulombTerm = -coulombConstant * Z * (Z - 1) * A ** (-1/3)
    asymmetryTerm = -asymmetryConstant * ((N - Z) ** 2) / A
    
    #Pairing term
    if A % 2 == 1:
        pairingTerm = 0
    elif Z % 2 == 0 and N % 2 == 0:
        pairingTerm = pairingConstant * A ** (-1/2)
    elif Z % 2 == 1 and N % 2 == 1:
        pairingTerm = -pairingConstant * A ** (-1/2)
    else:
        pairingTerm = 0
    output = (volumeTerm + surfaceTerm + coulombTerm + asymmetryTerm + pairingTerm) / A
    return(output)

## Simulated Annealing

In [13]:
class SimulatedAnnealing:
    def __init__(
        self,
        ranges
    ):
        self.ranges = ranges
    
    def optimize(self, objectiveFunction, numIterations = 25000, standardDeviation = 1, coolingFactor = 0.99):
        currentX = [np.mean(x) for x in self.ranges]
        currentY = objectiveFunction(currentX)
        heat = 1
        for i in range(numIterations):
            heat = heat * coolingFactor
            newX = [r.gauss(x, standardDeviation) for x in currentX]
            newY = objectiveFunction(newX)
            if newY > currentY:
                currentY = newY
                currentX = newX
            else:
                prob = r.uniform(0, 1)
                if prob > np.e ** -(currentY - newY)/(heat + (10 ** -10)):
                    currentY = newY
                    currentX = newX
        
        return(currentX)
sa = SimulatedAnnealing([[1, 120], [1, 180]])
print(sa.optimize(SEMF))

[np.float64(26.306735872197567), np.float64(32.76351426730145)]
